#### 1. 누적 매출액에 대한 회귀 분석 -> 전체 영화에 대한 회귀 분석

In [96]:
# 라이브러리 호출
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 그래프상에서 한글을 나타내기 위해 korean_mat 설치
!pip install koreanize-matplotlib
import koreanize_matplotlib

%config InlineBackend.figure_format = 'retina'

# 데이터 불러오기
df = pd.read_csv('/Users/gimdogyun/Desktop/빅데이터_애널리틱스_프로젝트/Big_analytics_project/movie_finished_toFDA_V9.csv')
df.head()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


,naver_movie_code,kobis_movie_code,movie_title,release_date,DoW_release,Date,DoW,MoY,D_D,DailyTotal,...,국적평균누적관객수,국적평균스크린수,등급평균관객수,등급평균누적관람객수,등급평균스크린수,7days_before,5days_before,Day_of_release,5days_after,7days_after
0,107374,20142641,나를 잊지 말아요,2016.1.7,Thursday,2016.1.7,Thursday,January,0,612,...,859493,95,9090,596341,79,11,11,13,6,33
1,107374,20142641,나를 잊지 말아요,2016.1.7,Thursday,2016.1.8,Friday,January,1,332,...,859493,95,9090,596341,79,11,11,13,6,33
2,107374,20142641,나를 잊지 말아요,2016.1.7,Thursday,2016.1.9,Saturday,January,2,394,...,859493,95,9090,596341,79,11,11,13,6,33
3,107374,20142641,나를 잊지 말아요,2016.1.7,Thursday,2016.1.10,Sunday,January,3,205,...,859493,95,9090,596341,79,11,11,13,6,33
4,107374,20142641,나를 잊지 말아요,2016.1.7,Thursday,2016.1.11,Monday,January,4,246,...,859493,95,9090,596341,79,11,11,13,6,33


#### 영화별로 기본 모형에 따라 어떤 것이 누적 매출액에 어떤 식으로 영향을 주는 지 살펴본다

* 기본 모형_1: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 : 영화의 내적 요인
* 기본 모형_2: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 + 스크린수 + 경쟁 영화 수  : 영화의 내적 요인 + 영화의 외부요인
* 기본 모형_3: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 + 스크린수 + 경쟁 영화 수 + 포털 평점 + 소셜미디어 : 영화의 내적 요인 + 영화의 외부요인 + 입소문 요인

#### 변수 설명
* 1. 누적 매출액: 2주간의 영화 매출액
* ----- 독립변수 ------
* ----영화의 내적 요인 --------
* 2. 국적: 영화가 어느 국적인가 -> 한국 영화인가? 혹은 외국 영화인가?
* 3. 장르: 영화의 장르 정보
* 4. 등급: 영화 -> 12세이하, 12세 이상 15세 ...
* 5. 감독: 감독 파워를 사용
* 6. 배우: 배우 파워를 사용
* ----영화의 외부 요인 --------
* 7. 첫째주 스크린 점유율 -> 개봉사 파워? 혹은 
* 8. 개봉 경쟁 영화 수
* ----영화의 입소문 요인-----
* 9. 실관람객 평점
* 10. 소셜미디어 정보

In [97]:
# 국적 칼럼
col_national = ['nationality']

# 장르 정보: 
col_genre= ['사극', 'SF', '어드벤처', '액션', '범죄', '판타지', '가족', '미스터리', '스릴러', '뮤지컬',
       '코미디', '공포(호러)', '애니메이션', '드라마', '서부극(웨스턴)', '전쟁', '멜로/로맨스', '기타',
       '다큐멘터리', '공연', '성인물(에로)']

# 관람 가능 연령 타입
col_watch_grade = ['watch_grade']

# 영화 감독 파워
col_direc_power = ['director_power']

# 영화 배우 파워
col_act_power = ['actor_power']

# 영화 상영 정보
col_screen = ['스크린점유율']

# 영화 평점 정보
col_score = ['RV_score_total']

# 영화의 경쟁 요인


# 소셜미디어 정보 -> 채영이가 만든 입소문 지수를 사용?

# movie_title
col_movie_title = ['movie_title']

# 종속변수
col_cum_money = ['누적_매출액']

#### 영화의 종속변수 -> 누적 매출액

In [98]:
# 1. 영화별 누적 매출액을 계산
sum_viewer = df.groupby('movie_title')['매출액'].sum()

# 2. 원본 데이터프레임에 누적 관객수를 매핑하여 새로운 컬럼으로 추가
df['누적_매출액'] = df['movie_title'].map(sum_viewer)

#### 1. 전체 영화에 대해서 누적 매출액에 대한 첫번째 모델 검증
* 기본 모형_1: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 : 영화의 내적 요인


In [99]:
# 전체 영화에 대한 첫번째 모델
df_model_1 = df[col_national+col_genre+col_watch_grade+col_direc_power+col_act_power+col_movie_title+col_cum_money].copy()
df_model_1

,nationality,사극,SF,어드벤처,액션,범죄,판타지,가족,미스터리,스릴러,...,멜로/로맨스,기타,다큐멘터리,공연,성인물(에로),watch_grade,director_power,actor_power,movie_title,누적_매출액
0,한국,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,15세이상관람가,449.614747,68205993.22,나를 잊지 말아요,3321622500
1,한국,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,15세이상관람가,449.614747,68205993.22,나를 잊지 말아요,3321622500
2,한국,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,15세이상관람가,449.614747,68205993.22,나를 잊지 말아요,3321622500
3,한국,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,15세이상관람가,449.614747,68205993.22,나를 잊지 말아요,3321622500
4,한국,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,15세이상관람가,449.614747,68205993.22,나를 잊지 말아요,3321622500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6130,미국,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,15세이상관람가,0.000000,0.00,애나벨 집으로,3513678610
6131,미국,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,15세이상관람가,0.000000,0.00,애나벨 집으로,3513678610
6132,미국,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,15세이상관람가,0.000000,0.00,애나벨 집으로,3513678610
6133,미국,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,15세이상관람가,0.000000,0.00,애나벨 집으로,3513678610


#### 분석을 위한 변수 전처리
* 국적 -> 만약 국적에 한국이 있다면 한국영화 -> 더미처리
* 장르 -> 더미 처리되어 있음
* 등급 -> 더미 처리
* 감독 파워 -> log 처리(영화마다 값의 차이가 너무 커서 log처리함)
* 배우 파워 -> log 처리(영화마다 값의 차이가 너무 커서 log처리함)


* 데이터 전처리를 위한 함수 정의

In [100]:
def create_dummies(df, columns, drop_first=True): # 더미 변수 생성을 위한 함수
    return pd.get_dummies(df, columns=columns, drop_first=drop_first)

def convert_data_types(df): # True,False -> 0,1로 변환
    df.update(df.select_dtypes(include=['bool']).astype(int))
    return df

def apply_log_transformation(df, columns): # 로그 변환
    for column in columns:
        df[column] = np.log1p(df[column])
    return df

def compute_daily_averages(df, daily_columns, key='movie_title'): # 일별 데이터 집계
    daily_averages = df.groupby(key)[daily_columns].mean().reset_index()
    df = df.drop(columns=daily_columns).merge(daily_averages, on=key, how='left')
    return df.groupby(key).mean().reset_index()

In [101]:
# 국적에 대해 한국인지 아닌지로 처리
# '한국영화'라는 값이 있으면 1로, 그렇지 않으면 0으로 변경 -> 1이면 한국영화
df_model_1['nationality'] = df_model_1['nationality'].apply(lambda x: '한국영화' if '한국' in x else '외국영화')

# 더미 변수 생성
df_model_1 = create_dummies(df_model_1, ['nationality', 'watch_grade'])

# 데이터 타입 변환
df_model_1 = convert_data_types(df_model_1)

# 로그 처리
log_columns = ['director_power', 'actor_power','누적_매출액']
df_model_1 = apply_log_transformation(df_model_1, log_columns)

/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 1 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly

In [102]:
df_model_1 = df_model_1.groupby('movie_title').mean()

#### 1. 첫번째 모델로 회귀 분석

In [103]:
import statsmodels.api as sm


# 독립변수(X)와 종속변수(y) 설정
X = df_model_1.drop('누적_매출액', axis=1)
y = df_model_1['누적_매출액']

# X에 상수항 추가 (절편 포함)
X = sm.add_constant(X)

# 회귀 모델 생성 및 적합
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 누적_매출액   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     5.733
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           2.56e-15
Time:                        04:06:32   Log-Likelihood:                -589.21
No. Observations:                 409   AIC:                             1230.
Df Residuals:                     383   BIC:                             1335.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   21.0750      0.345     61.026      0.000      20.396      21.754
사극                       0.4760      0.354      1.344      0.180      -0.220       1.172
SF                       0.0363      0.206      0.176      0.860      -0.369       0.441
어드벤처                     0.3436      0.169      2.028      0.043       0.010       0.677
액션                       0.5459      0.160      3.411      0.001       0.231       0.861
범죄                       0.5196      0.195      2.663      0.008       0.136       0.903
판타지                      0.1282      0.215      0.597      0.551      -0.294       0.551
가족                       0.0011      0.272      0.004      0.997      -0.534       0.536
미스터리                     0.0484      0.265      0.183      0.855      -0.473       0.570
스릴러                      0.2444      0.173      1.412      0.159      -0.096       0.585
뮤지컬                      1.2222      0.558      2.192      0.029       0.126       2.319
코미디                      0.3375      0.165      2.043      0.042       0.013       0.662
공포(호러)                   0.3747      0.259      1.445      0.149      -0.135       0.885
애니메이션                   -0.1498      0.275     -0.545      0.586      -0.690       0.391
드라마                     -0.1286      0.158     -0.816      0.415      -0.438       0.181
서부극(웨스턴)                 0.2556      1.073      0.238      0.812      -1.855       2.366
전쟁                       0.0900      0.621      0.145      0.885      -1.132       1.312
멜로/로맨스                  -0.3515      0.222     -1.587      0.113      -0.787       0.084
기타                   -8.144e-16   3.44e-16     -2.369      0.018   -1.49e-15   -1.38e-16
다큐멘터리                   -0.7806      0.494     -1.580      0.115      -1.752       0.191
공연                       0.4142      1.142      0.363      0.717      -1.831       2.659
성인물(에로)              -3.431e-16   2.51e-16     -1.367      0.172   -8.37e-16     1.5e-16
director_power           0.0231      0.009      2.535      0.012       0.005       0.041
actor_power              0.0488      0.020      2.494      0.013       0.010       0.087
nationality_한국영화         0.6924      0.147      4.725      0.000       0.404       0.981
watch_grade_15세이상관람가    -0.2750      0.146     -1.881      0.061      -0.562       0.012
watch_grade_전체관람가       -0.3238      0.252     -1.284      0.200      -0.820       0.172
watch_grade_청소년관람불가     -0.5908      0.204     -2.890      0.004      -0.993      -0.189
==============================================================================
Omnibus:                        5.944   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.051   Jarque-Bera (JB):                3.969
Skew:                           0.064   Prob(JB):                        0.137
Kurtosis

####  --> 영화의 내적 특성들로 회귀 모델을 돌릴 경우에 R_score가 너무 낮게 나오고, 일부 칼럼의 상관계수가 도메인 지식과 반대로 나옴(watch_grade)

#### 2. 두번째 모델
* 기본 모형_2: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 + 스크린수 + 경쟁 영화 수  : 영화의 내적 요인 + 영화의 외부요인

* 기본 모형_1과 달리, 스크린수와 경쟁영화수라는 칼럼이 추가되었다. -> 이는 영화의 외부요인이다
* 스크린수 -> 첫째주 스크린 점유율을 사용하였다. eda 결과 상영사와 배급사는 영화의 흥행 여부를 주단위로 예측해 영화의 스크린 점유율이 달라진다. 따라서 전체 스크린 점유율을 사용하는 것은 옳지 않다고 생각했다. 따라서 첫째주 스크린 점유율 평균을 사용하였다. 
* 영화의 경쟁 영화 수 -> 개봉일 기준 7일전 영화 수 + 5일전 영화 수 + 5일후 영화 수 + 7일후 영화 수 

In [104]:
# 영화마다 첫번째 일요일이 나올때까지 sum에 스크린 점유율을 합한다 -> 만약 영화의 개봉일이 목요일인 경우 sum + 목요일 스크린 점유율 + 금요일 스크린 점유율 + 토요일 스크린 점유율 + 일요일 스크린 점유율
# 일자를 정렬 순서로 설정
df[['movie_title','DoW','스크린점유율']]
# 요일 순서 지정
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['DoW'] = pd.Categorical(df['DoW'], categories=day_order, ordered=True)

# 결과 저장용 리스트
results = []

# 영화별로 그룹화
for movie, group in df.groupby('movie_title'):
    # 첫 번째 일요일의 인덱스 찾기
    first_sunday_idx = group[group['DoW'] == 'Sunday'].index.min()
    
    # 첫 번째 일요일까지 데이터 필터링
    filtered_group = group.loc[:first_sunday_idx]
    
    # 스크린 점유율 평균 계산
    avg_screen_share = filtered_group['스크린점유율'].mean()
    
    # 결과 저장
    results.append({'movie_title': movie, '1주차스크린점유율': avg_screen_share})

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)
result_df[result_df['movie_title'] == '군함도']

# df와 병합
df= df.merge(result_df, on='movie_title', how='left')



In [105]:
# 영화의 경쟁영화 수 -> 개봉일 기준 7일전 영화 수 + 5일전 영화 수 + 5일후 영화 수 + 7일후 영화 수
df['2주동안의 경쟁영화수'] = df['7days_before'] + df['5days_before'] + df['Day_of_release'] + df['5days_after'] + df['7days_after']

In [106]:
# 새로운 칼럼
# 1주차 스크린 점유율
screen_col = ['1주차스크린점유율']

# 경쟁 영화 수
compete_col = ['2주동안의 경쟁영화수']

In [107]:
df_model_2 = df[col_national+col_genre+col_watch_grade+col_direc_power+col_act_power+screen_col+compete_col+col_movie_title+col_cum_money].copy()

* 모델을 돌리기 위해 데이터 전처리

In [108]:
# 국적에 대해 한국인지 아닌지로 처리
# '한국영화'라는 값이 있으면 1로, 그렇지 않으면 0으로 변경 -> 1이면 한국영화
df_model_2['nationality'] = df_model_2['nationality'].apply(lambda x: '한국영화' if '한국' in x else '외국영화')

# 더미 변수 생성
df_model_2 = create_dummies(df_model_2, ['nationality', 'watch_grade'])

# 데이터 타입 변환
df_model_2 = convert_data_types(df_model_2)

# 로그 처리
log_columns = ['director_power', 'actor_power','2주동안의 경쟁영화수','누적_매출액']
df_model_2 = apply_log_transformation(df_model_2, log_columns)

df_model_2 = df_model_2.groupby('movie_title').mean()

/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 1 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly

In [109]:
import statsmodels.api as sm


# 독립변수(X)와 종속변수(y) 설정
X = df_model_2.drop('누적_매출액', axis=1)
y = df_model_2['누적_매출액']

# X에 상수항 추가 (절편 포함)
X = sm.add_constant(X)

# 회귀 모델 생성 및 적합
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 누적_매출액   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     41.28
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           5.42e-96
Time:                        04:06:33   Log-Likelihood:                -374.58
No. Observations:                 409   AIC:                             805.2
Df Residuals:                     381   BIC:                             917.5
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   21.3099      0.826     25.803      0.000      19.686      22.934
사극                       0.1760      0.210      0.837      0.403      -0.238       0.590
SF                      -0.1738      0.122     -1.419      0.157      -0.414       0.067
어드벤처                    -0.1056      0.102     -1.034      0.302      -0.306       0.095
액션                      -0.0291      0.097     -0.299      0.765      -0.221       0.162
범죄                       0.1999      0.116      1.717      0.087      -0.029       0.429
판타지                     -0.1085      0.129     -0.844      0.399      -0.361       0.144
가족                       0.0481      0.161      0.298      0.766      -0.269       0.365
미스터리                     0.1216      0.157      0.773      0.440      -0.188       0.431
스릴러                      0.0723      0.103      0.703      0.483      -0.130       0.275
뮤지컬                      0.1793      0.333      0.538      0.591      -0.476       0.835
코미디                      0.1422      0.098      1.447      0.149      -0.051       0.336
공포(호러)                   0.1612      0.154      1.046      0.296      -0.142       0.464
애니메이션                    0.0334      0.163      0.204      0.838      -0.288       0.354
드라마                     -0.0444      0.094     -0.475      0.635      -0.228       0.140
서부극(웨스턴)                 0.8459      0.638      1.325      0.186      -0.409       2.101
전쟁                       0.3076      0.369      0.834      0.405      -0.418       1.033
멜로/로맨스                  -0.0397      0.132     -0.301      0.764      -0.300       0.220
기타                   -8.649e-14   9.76e-15     -8.863      0.000   -1.06e-13   -6.73e-14
다큐멘터리                   -0.0367      0.295     -0.125      0.901      -0.616       0.543
공연                       1.2706      0.678      1.874      0.062      -0.063       2.604
성인물(에로)               5.881e-14   5.98e-15      9.831      0.000     4.7e-14    7.06e-14
director_power          -0.0005      0.005     -0.091      0.927      -0.011       0.010
actor_power              0.0171      0.012      1.446      0.149      -0.006       0.040
1주차스크린점유율               14.9442      0.562     26.590      0.000      13.839      16.049
2주동안의 경쟁영화수             -0.2093      0.166     -1.257      0.209      -0.537       0.118
nationality_한국영화         0.0013      0.091      0.014      0.988      -0.177       0.180
watch_grade_15세이상관람가    -0.1797      0.087     -2.070      0.039      -0.350      -0.009
watch_grade_전체관람가       -0.2209      0.150     -1.476      0.141      -0.515       0.073
watch_grade_청소년관람불가     -0.2466      0.122     -2.014      0.045      -0.487      -0.006
==============================================================================
Omnibus:                        0.966   Durbin-Watson:             

#### 기본 모형_3: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 + 스크린수 + 경쟁 영화 수 + 포털 평점 + 소셜미디어 : 영화의 내적 요인 + 영화의 외부요인 + 입소문 요인

* 기존의 모형에서 입소문 요인을 추가하였다. -> 1주차 실관람객 평점

In [110]:
# 영화의 1주차_실관람객 평점을 입소문 요인으로서 추가하였다
# 영화마다 첫번째 일요일이 나올때까지 sum에 실관람객 평점을 합한다 -> 만약 영화의 개봉일이 목요일인 경우 sum + 목요일 스크린 점유율 + 금요일 스크린 점유율 + 토요일 스크린 점유율 + 일요일 스크린 점유율
# 일자를 정렬 순서로 설정
df[['movie_title','DoW','RV_Dailyscore']]
# 요일 순서 지정
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['DoW'] = pd.Categorical(df['DoW'], categories=day_order, ordered=True)

# 결과 저장용 리스트
results = []

# 영화별로 그룹화
for movie, group in df.groupby('movie_title'):
    # 첫 번째 일요일의 인덱스 찾기
    first_sunday_idx = group[group['DoW'] == 'Sunday'].index.min()
    
    # 첫 번째 일요일까지 데이터 필터링
    filtered_group = group.loc[:first_sunday_idx]
    
    # 실관람객 평균 계산
    avg_daily_score = filtered_group['RV_Dailyscore'].mean()
    
    # 결과 저장
    results.append({'movie_title': movie, '1주차실관람객평균평점': avg_daily_score})

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# df와 병합
df= df.merge(result_df, on='movie_title', how='left')

* 1주차실관람객평균평점 -> nan값이 세개 존재 -> 관람객 평균 평점(NTZ_Dailyscore)으로 대체
* 나의 소녀시대,업그레이드,러브 유어셀프 인 서울

In [111]:
 df['1주차실관람객평균평점'].fillna(99999,inplace=True)
df[df['1주차실관람객평균평점']==99999][['NTZ_Dailyscore','movie_title']]

df[df['movie_title'] == '나의 소녀시대'][['NTZ_Dailyscore','DoW']] # 9.4
df[df['movie_title'] == '업그레이드'][['NTZ_Dailyscore','DoW']]# 8.8
df[df['movie_title'] == '러브 유어셀프 인 서울'][['NTZ_Dailyscore','DoW']] # 9.3

replace_dic = {'나의 소녀시대':9.4,'업그레이드':8.8,'러브 유어셀프 인 서울':9.3}




/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1925439567.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['1주차실관람객평균평점'].fillna(99999,inplace=True)


* 입소문 요인 칼럼 -> 1주차실관람객평균평점
* 모델 3: 기본 모형_3: 누적 매출액 = 국적 + 장르 + 등급 + 감독 + 배우 + 스크린수 + 경쟁 영화 수 + 포털 평점 + 소셜미디어 : 영화의 내적 요인 + 영화의 외부요인 + 입소문 요인

In [112]:
score_col = ['1주차실관람객평균평점']

In [113]:
# null 처리: 실관람객 평점이 존재하지 않은 3개의 영화에 대해서는 네티즌 평균 평점(1주차)로 처리
# NaN 값을 영화별로 특정 값으로 채우기
df.loc[df['movie_title'] == '나의 소녀시대', '1주차실관람객평균평점'] = 9.4
df.loc[df['movie_title'] == '업그레이드', '1주차실관람객평균평점'] = 8.8
df.loc[df['movie_title'] == '러브 유어셀프 인 서울', '1주차실관람객평균평점'] = 9.3





In [114]:
df_model_3 = df[col_national+col_genre+col_watch_grade+col_direc_power+col_act_power+screen_col+compete_col+score_col+col_movie_title+col_cum_money].copy()

* 모델 생성을 위한 전처리

In [115]:
# 국적에 대해 한국인지 아닌지로 처리
# '한국영화'라는 값이 있으면 1로, 그렇지 않으면 0으로 변경 -> 1이면 한국영화
df_model_3['nationality'] = df_model_3['nationality'].apply(lambda x: '한국영화' if '한국' in x else '외국영화')

# 더미 변수 생성
df_model_3 = create_dummies(df_model_3, ['nationality', 'watch_grade'])

# 데이터 타입 변환
df_model_3 = convert_data_types(df_model_3)

# 로그 처리
log_columns = ['director_power', 'actor_power','2주동안의 경쟁영화수','1주차실관람객평균평점','누적_매출액']
df_model_3 = apply_log_transformation(df_model_3, log_columns)

df_model_3 = df_model_3.groupby('movie_title').mean()

/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 1 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly

In [116]:
# 독립변수(X)와 종속변수(y) 설정
X = df_model_3.drop('누적_매출액', axis=1)
y = df_model_3['누적_매출액']

# X에 상수항 추가 (절편 포함)
X = sm.add_constant(X)

# 회귀 모델 생성 및 적합
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 누적_매출액   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     44.65
Date:                Sun, 08 Dec 2024   Prob (F-statistic):          2.37e-102
Time:                        04:06:34   Log-Likelihood:                -356.40
No. Observations:                 409   AIC:                             770.8
Df Residuals:                     380   BIC:                             887.2
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.4931      1.133     14.563      0.000      14.266      18.720
사극                       0.2370      0.202      1.175      0.241      -0.160       0.634
SF                      -0.1001      0.118     -0.849      0.396      -0.332       0.132
어드벤처                    -0.0922      0.098     -0.943      0.346      -0.285       0.100
액션                      -0.0156      0.093     -0.167      0.867      -0.199       0.168
범죄                       0.1973      0.111      1.770      0.078      -0.022       0.416
판타지                     -0.0680      0.123     -0.551      0.582      -0.311       0.175
가족                       0.0258      0.155      0.167      0.868      -0.278       0.330
미스터리                     0.1298      0.151      0.861      0.390      -0.166       0.426
스릴러                      0.0897      0.099      0.910      0.364      -0.104       0.284
뮤지컬                      0.1087      0.319      0.340      0.734      -0.519       0.737
코미디                      0.1450      0.094      1.540      0.124      -0.040       0.330
공포(호러)                   0.3098      0.150      2.069      0.039       0.015       0.604
애니메이션                   -0.0232      0.157     -0.148      0.883      -0.331       0.285
드라마                     -0.1236      0.091     -1.365      0.173      -0.302       0.054
서부극(웨스턴)                 0.8890      0.611      1.454      0.147      -0.313       2.091
전쟁                       0.2364      0.353      0.669      0.504      -0.458       0.931
멜로/로맨스                  -0.0813      0.127     -0.641      0.522      -0.331       0.168
기타                   -3.954e-15   4.29e-16     -9.213      0.000    -4.8e-15   -3.11e-15
다큐멘터리                   -0.3301      0.287     -1.152      0.250      -0.894       0.233
공연                       1.0910      0.650      1.678      0.094      -0.187       2.369
성인물(에로)               9.956e-16   1.81e-16      5.513      0.000    6.41e-16    1.35e-15
director_power          -0.0006      0.005     -0.107      0.915      -0.011       0.010
actor_power              0.0207      0.011      1.830      0.068      -0.002       0.043
1주차스크린점유율               14.7064      0.540     27.245      0.000      13.645      15.768
2주동안의 경쟁영화수             -0.1249      0.160     -0.781      0.436      -0.440       0.190
1주차실관람객평균평점              1.9478      0.328      5.943      0.000       1.303       2.592
nationality_한국영화         0.0780      0.088      0.887      0.375      -0.095       0.251
watch_grade_15세이상관람가    -0.1257      0.084     -1.503      0.134      -0.290       0.039
watch_grade_전체관람가       -0.2404      0.143     -1.676      0.095      -0.522       0.042
watch_grade_청소년관람불가     -0.1314      0.119     -1.105      0.270      -0.365       0.102
=========================================================

#### 후진 제거법을 사용하여 모델을 돌려보기 -> 독립 변수는 모델 3을 사용

In [117]:
import pandas as pd
import statsmodels.api as sm
import itertools
import time

# 변수 선택 과정을 수행하는 함수
def processSubset(X, y, feature_set):
    model = sm.OLS(y, sm.add_constant(X[list(feature_set)]))
    regression = model.fit()
    return {"model": regression, "AIC": regression.aic}

# 후진제거법
def backward(X, y, predictors):
    results = []
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X, y, combo))
    models = pd.DataFrame(results)
    best_model = models.loc[models['AIC'].argmin()]
    return best_model

# 후진제거법을 사용한 전체 모델 선택
def backward_model(X, y):
    predictors = X.columns.tolist()
    Bmodels = pd.DataFrame(columns=["AIC", "model"], index = range(1, len(X.columns)))
    
    while len(predictors) > 1:
        current_model = backward(X, y, predictors)
        Bmodels.loc[len(predictors) - 1] = current_model
        predictors = current_model['model'].model.exog_names
        predictors = [p for p in predictors if p != 'const']  # 상수항 제외
        
    # 최적의 모델 추출
    best_model_index = Bmodels['AIC'].idxmin()
    best_model = Bmodels.loc[best_model_index]
    return best_model

In [118]:
# 데이터 준비
# 독립변수(X)와 종속변수(y) 설정
X = df_model_3.drop('누적_매출액', axis=1)
y = df_model_3['누적_매출액']

# 후진제거법을 이용한 최적 모델 탐색
final_model = backward_model(X, y)

final_model['model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 누적_매출액   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     139.3
Date:                Sun, 08 Dec 2024   Prob (F-statistic):          2.93e-117
Time:                        04:07:08   Log-Likelihood:                -363.63
No. Observations:                 409   AIC:                             747.3
Df Residuals:                     399   BIC:                             787.4
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                15.8666      0.714     22.217      0.000      14.463      17.271
범죄                    0.2060      0.101      2.031      0.043       0.007       0.405
코미디                   0.1545      0.087      1.776      0.076      -0.016       0.325
공포(호러)                0.3922      0.134      2.935      0.004       0.129       0.655
서부극(웨스턴)              0.8735      0.598      1.461      0.145      -0.302       2.049
공연                    1.3397      0.617      2.173      0.030       0.128       2.552
actor_power           0.0288      0.010      2.897      0.004       0.009       0.048
1주차스크린점유율            14.8266      0.471     31.490      0.000      13.901      15.752
1주차실관람객평균평점           1.8525      0.302      6.129      0.000       1.258       2.447
watch_grade_전체관람가    -0.1655      0.079     -2.091      0.037      -0.321      -0.010
==============================================================================
Omnibus:                        0.066   Durbin-Watson:                   1.877
Prob(Omnibus):                  0.967   Jarque-Bera (JB):                0.138
Skew:                          -0.024   Prob(JB):                        0.933
Kurtosis:                       2.924   Cond. No.                         421.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 종속변수를 수익성으로 하고, 모델 3의 변수를 사용하여 후진 제거법을 사용

* 수익성 칼럼 만들기

In [119]:
# 15번째 행 간격으로 데이터 필터링
filtered_df = df.iloc[::15]

# '수익성' 계산
filtered_df['수익성'] = filtered_df['누적_매출액'] / filtered_df['좌석수']

# 계산된 '수익성'을 원래 데이터프레임으로 매핑
df['수익성'] = None  # 초기화
df.loc[filtered_df.index, '수익성'] = filtered_df['수익성']

/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/2342551628.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['수익성'] = filtered_df['누적_매출액'] / filtered_df['좌석수']


In [120]:
# 'movie_title'별 첫 번째 '수익성' 값을 그룹의 모든 행에 적용
df['수익성'] = df.groupby('movie_title')['수익성'].transform('first')

# 결과 확인
df[['movie_title', '누적_매출액', '좌석수', '수익성']]

,movie_title,누적_매출액,좌석수,수익성
0,나를 잊지 말아요,3321622500,360219,9221.119652
1,나를 잊지 말아요,3321622500,359876,9221.119652
2,나를 잊지 말아요,3321622500,373884,9221.119652
3,나를 잊지 말아요,3321622500,347741,9221.119652
4,나를 잊지 말아요,3321622500,336975,9221.119652
...,...,...,...,...
6130,애나벨 집으로,3513678610,39894,12418.458366
6131,애나벨 집으로,3513678610,42355,12418.458366
6132,애나벨 집으로,3513678610,49081,12418.458366
6133,애나벨 집으로,3513678610,50664,12418.458366


In [121]:
# 수익성
profit_col = ['수익성']

In [122]:
df_model_3 = pd.merge(df_model_3,df[['movie_title','수익성']],on='movie_title')

In [123]:
df_model_3 = df_model_3.groupby('movie_title').mean()
df_model_3['수익성'] = pd.to_numeric(df_model_3['수익성'], errors='coerce')
df_model_3['수익성']= np.log1p(df_model_3['수익성'])

In [124]:
# 데이터 준비
# 독립변수(X)와 종속변수(y) 설정
X = df_model_3.drop(['누적_매출액','수익성'], axis=1)
y = df_model_3['수익성']

# 후진제거법을 이용한 최적 모델 탐색
final_model = backward_model(X, y)

final_model['model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    수익성   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     25.58
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           7.17e-32
Time:                        04:07:42   Log-Likelihood:                -356.76
No. Observations:                 409   AIC:                             731.5
Df Residuals:                     400   BIC:                             767.6
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.4174      0.999      5.425      0.000       3.454       7.381
액션             -0.1438      0.069     -2.100      0.036      -0.278      -0.009
범죄              0.1457      0.100      1.463      0.144      -0.050       0.341
공포(호러)          0.2200      0.124      1.768      0.078      -0.025       0.465
애니메이션           0.5285      0.084      6.294      0.000       0.363       0.694
기타           1.126e-14    1.3e-15      8.686      0.000    8.71e-15    1.38e-14
다큐멘터리           0.3679      0.247      1.492      0.136      -0.117       0.853
성인물(에로)      1.254e-15   1.43e-16      8.740      0.000    9.72e-16    1.54e-15
1주차스크린점유율       3.9792      0.465      8.554      0.000       3.065       4.894
2주동안의 경쟁영화수    -0.3919      0.151     -2.594      0.010      -0.689      -0.095
1주차실관람객평균평점     2.4579      0.297      8.264      0.000       1.873       3.043
==============================================================================
Omnibus:                        0.893   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.640   Jarque-Bera (JB):                0.685
Skew:                           0.073   Prob(JB):                        0.710
Kurtosis:                       3.137   Cond. No.                     4.02e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.03e-36. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### 현재까지는 모든 수치형 변수들에 대해 로그 변환을 해왔디 -> but 변수마다 다른 스케일링을 사용한다면?

* 후진 제거법을 하기 위한 + 각각의 다른 스케일릴을 위한 df 만들기

In [125]:
df_model_back = df[col_national+col_genre+col_watch_grade+col_direc_power+col_act_power+screen_col+compete_col+score_col+col_movie_title+col_cum_money+profit_col].copy()

* 분포가 정규분포가 아닌 것들에 대해서만 log변환을 취한다.

In [126]:
log_columns = ['director_power', 'actor_power','누적_매출액','수익성']

df_model_back['수익성'] = pd.to_numeric(df_model_back['수익성'], errors='coerce')

df_model_back = apply_log_transformation(df_model_back, log_columns)

* 다른 변수들에 대한 변환

In [127]:
# 국적에 대해 한국인지 아닌지로 처리
# '한국영화'라는 값이 있으면 1로, 그렇지 않으면 0으로 변경 -> 1이면 한국영화
df_model_back['nationality'] = df_model_back['nationality'].apply(lambda x: '한국영화' if '한국' in x else '외국영화')

# 더미 변수 생성
df_model_back = create_dummies(df_model_back, ['nationality', 'watch_grade'])

# 데이터 타입 변환
df_model_back = convert_data_types(df_model_back)

df_model_back = df_model_back.groupby('movie_title').mean()

/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 1 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.update(df.select_dtypes(include=['bool']).astype(int))
/var/folders/mz/tk5z94ds1qd9c1jb2p8qys9m0000gn/T/ipykernel_95239/1031698659.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly

In [128]:
df_model_back

,사극,SF,어드벤처,액션,범죄,판타지,가족,미스터리,스릴러,뮤지컬,...,actor_power,1주차스크린점유율,2주동안의 경쟁영화수,1주차실관람객평균평점,누적_매출액,수익성,nationality_한국영화,watch_grade_15세이상관람가,watch_grade_전체관람가,watch_grade_청소년관람불가
movie_title,,,,,,,,,,,,,,,,,,,,,
0.0MHz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.311211,0.070600,153.0,5.587778,20.771733,9.045517,1.0,1.0,0.0,0.0
12 솔져스,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.378307,0.029200,115.0,9.100000,21.280387,9.443582,0.0,1.0,0.0,0.0
1987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.691708,0.209400,73.0,9.206219,24.320857,10.437006,1.0,1.0,0.0,0.0
1급기밀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.493524,0.073800,135.0,8.989677,21.219533,8.995740,1.0,0.0,0.0,0.0
23 아이덴티티,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,15.422221,0.118400,89.0,7.875880,23.269788,10.583162,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
혹성탈출: 종의 전쟁,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.634275,0.157667,116.0,8.252512,23.514681,9.843569,0.0,0.0,0.0,0.0
혼숨,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.631074,0.100400,85.0,6.493778,21.515367,9.831606,1.0,1.0,0.0,0.0
후드,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.075473,0.102000,137.0,7.074703,21.487938,8.860007,0.0,0.0,0.0,0.0


#### 1. 종속변수를 log(누적_매출액)로 하여 후진 제거법

In [129]:
# 데이터 준비
# 독립변수(X)와 종속변수(y) 설정
X = df_model_back.drop(['누적_매출액','수익성'], axis=1)
y = df_model_back['누적_매출액']

# 후진제거법을 이용한 최적 모델 탐색
final_model = backward_model(X, y)

final_model['model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 누적_매출액   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     115.1
Date:                Sun, 08 Dec 2024   Prob (F-statistic):          4.10e-116
Time:                        04:08:16   Log-Likelihood:                -361.24
No. Observations:                 409   AIC:                             746.5
Df Residuals:                     397   BIC:                             794.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                18.0904      0.357     50.711      0.000      17.389      18.792
사극                    0.3062      0.194      1.578      0.115      -0.075       0.688
범죄                    0.2227      0.102      2.193      0.029       0.023       0.422
스릴러                   0.1334      0.086      1.554      0.121      -0.035       0.302
코미디                   0.1784      0.088      2.034      0.043       0.006       0.351
공포(호러)                0.3783      0.135      2.804      0.005       0.113       0.644
서부극(웨스턴)              0.9240      0.596      1.550      0.122      -0.248       2.096
공연                    1.3322      0.615      2.168      0.031       0.124       2.541
actor_power           0.0291      0.010      2.919      0.004       0.009       0.049
1주차스크린점유율            14.7496      0.471     31.293      0.000      13.823      15.676
1주차실관람객평균평점           0.2246      0.035      6.380      0.000       0.155       0.294
watch_grade_전체관람가    -0.1482      0.080     -1.853      0.065      -0.305       0.009
==============================================================================
Omnibus:                        0.120   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.942   Jarque-Bera (JB):                0.230
Skew:                          -0.006   Prob(JB):                        0.891
Kurtosis:                       2.884   Cond. No.                         375.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 2. 종속변수를 log(수익성)으로 하여 후진제거법

In [130]:
# 데이터 준비
# 독립변수(X)와 종속변수(y) 설정
X = df_model_back.drop(['누적_매출액','수익성'], axis=1)
y = df_model_back['수익성']

# 후진제거법을 이용한 최적 모델 탐색
final_model = backward_model(X, y)

final_model['model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    수익성   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.326
Method:                 Least Squares   F-statistic:                     29.17
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           1.71e-32
Time:                        04:08:49   Log-Likelihood:                -357.07
No. Observations:                 409   AIC:                             730.1
Df Residuals:                     401   BIC:                             762.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           6.9994      0.341     20.552      0.000       6.330       7.669
액션             -0.1444      0.068     -2.111      0.035      -0.279      -0.010
범죄              0.1481      0.100      1.488      0.137      -0.048       0.344
공포(호러)          0.2232      0.125      1.793      0.074      -0.022       0.468
애니메이션           0.5070      0.083      6.073      0.000       0.343       0.671
1주차스크린점유율       3.9162      0.463      8.467      0.000       3.007       4.825
2주동안의 경쟁영화수    -0.0035      0.001     -2.477      0.014      -0.006      -0.001
1주차실관람객평균평점     0.2943      0.034      8.642      0.000       0.227       0.361
==============================================================================
Omnibus:                        0.375   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.829   Jarque-Bera (JB):                0.222
Skew:                           0.037   Prob(JB):                        0.895
Kurtosis:                       3.087   Cond. No.                     1.76e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### 영화의 장르 정보를 없애고 => 대중적 장르성 정보를 포힘

In [131]:
df_genre = pd.read_csv('/Users/gimdogyun/Desktop/빅데이터_애널리틱스_프로젝트/Big_analytics_project/genre_score.csv')
df_genre

,movie_title,genre_score
0,0.0MHz,0
1,12 솔져스,2
2,1987,1
3,1급기밀,1
4,23 아이덴티티,0
...,...,...
404,혹성탈출: 종의 전쟁,3
405,혼숨,0
406,후드,1
407,희생부활자,0


In [144]:
#df_model_back의 movie_title 인덱스에 따라 df_genre의 movie_title 칼럼과 합치기
df_model_back = pd.merge(df_model_back,df_genre,on='movie_title')
df_model_back_genre = df_model_back.drop(col_genre,axis=1).copy()


In [147]:
# 데이터 준비
# 독립변수(X)와 종속변수(y) 설정
X = df_model_back_genre.drop(['누적_매출액','수익성','movie_title'], axis=1)
y = df_model_back_genre['수익성']

# 후진제거법을 이용한 최적 모델 탐색
final_model = backward_model(X, y)

final_model['model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    수익성   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     36.30
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           1.10e-30
Time:                        04:14:48   Log-Likelihood:                -365.20
No. Observations:                 409   AIC:                             742.4
Df Residuals:                     403   BIC:                             766.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 7.1551      0.336     21.324      0.000       6.495       7.815
1주차스크린점유율             3.7414      0.454      8.248      0.000       2.850       4.633
2주동안의 경쟁영화수          -0.0038      0.001     -2.691      0.007      -0.007      -0.001
1주차실관람객평균평점           0.2951      0.034      8.701      0.000       0.228       0.362
watch_grade_전체관람가     0.3542      0.079      4.485      0.000       0.199       0.509
genre_score_x        -0.0933      0.039     -2.373      0.018      -0.171      -0.016
==============================================================================
Omnibus:                        0.035   Durbin-Watson:                   1.826
Prob(Omnibus):                  0.983   Jarque-Bera (JB):                0.077
Skew:                          -0.022   Prob(JB):                        0.962
Kurtosis:                       2.949   Cond. No.                     1.70e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.7e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""